In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
df = pd.read_csv('2020-4-27.csv')
df.head()

,Title,Address,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Year,Views,Area,Road,Road Width,Road Type,Build Area,Posted,Amenities
0,Flat System House On Sale At Budhanikantha : H...,"Budhanikantha, Budhanilkantha, Kathmandu",Kathmandu,90000000,6,3,2.0,10,West,2073.0,17,1-0-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,18-0-0-0 Aana,11 hours ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
1,21 Aana Beautiful House : House For Sale In Bu...,"Pasikot, Budhanilkantha, Kathmandu",Kathmandu,80000000,5,3,2.0,9,East,2073.0,26,0-21-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-14-0-0 Aana,12 hours ago,"['Parking', 'Garage', 'Fencing', 'Frontyard', ..."
2,Beautiful To Floor House : House For Sale In D...,"Dhapasi, Dhapasi, Kathmandu",Kathmandu,70000000,5,3,2.0,12,East,2071.0,154,1-1-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,10-1-0-0 Aana,2 days ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
3,Modern Architecture Bungalow House : House For...,"baluwatar, Baluwatar, Kathmandu",Kathmandu,130000000,6,4,3.0,9,West,2017.0,17,0-10-1-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-7-1-0 Aana,1 day ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
4,Modern Bungalow House : House For Sale In Golf...,"Golfutar, Golfutar, Kathmandu",Kathmandu,100000000,6,3,2.0,10,East,2070.0,59,0-19-2-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-14-2-0 Aana,2 days ago,"['Parking', 'Lawn', 'Garage', 'Balcony', 'Back..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2211 entries, 0 to 2210
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Title       2211 non-null   object 
 1   Address     2211 non-null   object 
 2   City        2211 non-null   object 
 3   Price       2211 non-null   int64  
 4   Bedroom     2211 non-null   int64  
 5   Bathroom    2211 non-null   int64  
 6   Floors      1039 non-null   float64
 7   Parking     2211 non-null   int64  
 8   Face        2211 non-null   object 
 9   Year        582 non-null    float64
 10  Views       2211 non-null   object 
 11  Area        2211 non-null   object 
 12  Road        2211 non-null   object 
 13  Road Width  2211 non-null   object 
 14  Road Type   1426 non-null   object 
 15  Build Area  2211 non-null   object 
 16  Posted      2211 non-null   object 
 17  Amenities   2211 non-null   object 
dtypes: float64(2), int64(4), object(12)
memory usage: 311.0+ KB


In [9]:
null_counts = df.isnull().sum()
null_counts

Title            0
Address          0
City             0
Price            0
Bedroom          0
Bathroom         0
Floors        1172
Parking          0
Face             0
Year          1629
Views            0
Area             0
Road             0
Road Width       0
Road Type      785
Build Area       0
Posted           0
Amenities        0
dtype: int64

### Columns not used:
- title
- floors : 1172 out of 2211 values missing
- year : 1629 out of 2211 values missing
- road : dependent on road width and road type
- posted
- amenities: 1067 out of 2211 values missing

In [19]:
sel_cols = ['Address','City','Bedroom','Bathroom','Parking','Face','Views','Area','Road Width','Road Type','Build Area',]
X = df[sel_cols].copy()
y = df['Price']
print(X.tail())
print(y)

                                     Address       City  Bedroom  Bathroom  \
2206             Bhangal, Chapali, Kathmandu  Kathmandu        5         5   
2207          Nagarjun, Sitapaila, Kathmandu  Kathmandu        4         2   
2208  Deuja Chowk, Budhanilkantha, Kathmandu  Kathmandu        5         3   
2209       Bhimsengola, Baneshwor, Kathmandu  Kathmandu        9         3   
2210        Gocha marg, Baluwatar, Kathmandu  Kathmandu        5         2   

      Parking        Face  Views      Area Road Width Road Type    Build Area  
2206        5  North East   3.1K  1 Ropani    16 Feet       NaN  N/A Sq. Feet  
2207        1  North East   1.4K  2.5 Aana    13 Feet       NaN  N/A Sq. Feet  
2208        1        East   1.5K    8 Aana    12 Feet       NaN  N/A Sq. Feet  
2209        4       North   2.5K   51 Aana    13 Feet       NaN  N/A Sq. Feet  
2210        5        East  19.6K   13 Aana    12 Feet       NaN       13 Aana  
0        90000000
1        80000000
2        700000

In [ ]:
# keep the middle values of the address columns only

#convert views like 1k to 1000

#convert all area and build area in squre feet